In [6]:
using Pkg
Pkg.activate(".")
Pkg.add("Zygote")
Pkg.add("MLUtils")
Pkg.add("Optimisers")
Pkg.add("ChainRulesCore")
Pkg.add("ImageMorphology")
Pkg.add("Plots")

using Lux, Random, NNlib, Zygote, LuxCUDA, CUDA, JLD2, DICOM
using Images
using MLUtils
using Optimisers
using ImageMorphology, ChainRulesCore, Statistics
using Plots

  Activating project at `~/Code/Lab/Observer`
   Resolving package versions...
  No Changes to `~/Code/Lab/Observer/Project.toml`
  No Changes to `~/Code/Lab/Observer/Manifest.toml`
Precompiling project...
  ✓ XSLT_jll
  ✓ BangBang → BangBangStructArraysExt
  ✓ Xorg_libxcb_jll
  ✓ MicroCollections
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libX11_jll
  ✓ OpenSSL
  ✓ Xorg_libXext_jll
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_libXrender_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXrandr_jll
  ✓ Xorg_libXinerama_jll
  ✓ Libglvnd_jll
  ✓ Xorg_libXi_jll
  ✓ Xorg_libXcursor_jll
  ✓ Xorg_xcb_util_cursor_jll
  ✓ Xorg_xkbcomp_jll
  ✓ GLFW_jll
  ✓ Xorg_xkeyboard_config_jll
  ✓ xkbcommon_jll
  ✓ Vulkan_Loader_jll
  ✓ Transducers
  ✓ HTTP
  ✓ FLoops
  ? Cairo_jll
  ✓ Qt6Base_jll
  ✓ MLUtils
  ✓ GR_jll
  ✓ GR
  ✓ Plots
  ✓ Plots → UnitfulExt
  35 dependencies successfully precompiled in 59 seconds. 224 already

In [8]:

# Rank(similar to threadID) of the current process.
dev = gpu_device()
dev_cpu = cpu_device()

println("num threads: $(Threads.nthreads())")

CUDA.allowscalar(false)

# dataset_dir = "/media/molloi-lab/2TB2/BAC ver3/subset2"
# train_dir = joinpath(dataset_dir, "train")
# test_dir = joinpath(dataset_dir, "test")

# # data_dir = "/media/molloi-lab/2TB/BAC/BAC_subset_but_full_image"

# V_P_dict = Dict("L CC" => 1, "L MLO" => 2, "R CC" => 3, "R MLO" => 4)

# train_SIDs = readdir(train_dir)
# test_SIDs = readdir(test_dir);

num threads: 8


In [9]:
function GetCalibConstant(serialNumber, thickness, kVp, anodeMaterial)
    CalibConstant = 1

    if serialNumber == "207924715888" #SantaClara Rm 2_20120905
        x = thickness - 6
        y = kVp
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                if y <= 25
                    CalibConstant = 90.30407
                else
                    if abs(x - 15) <= abs(x - 20)
                        CalibConstant = 85.54987
                    else
                        CalibConstant = 83.2794
                    end
                end

                return CalibConstant
            else
                if y <= 26
                    CalibConstant = 72.29046
                else
                    if abs(x - 30) <= abs(x - 35)
                        CalibConstant = 69.5362
                    else
                        CalibConstant = 67.34609
                    end
                end

                return CalibConstant
            end
        else
            a1 = 1.08075
            a2 = 11.52868
            b1 = -0.00525
            b2 = -0.05749
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "H1KRHR84e2e868"  # SanFrancisco Rm1
        x = thickness - 11
        y = kVp
        a1 = -75.05777275
        a2 = 199.5341025
        b1 = 0.027747672
        b2 = -0.153130287
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84e2e88e"  #SanFrancisco Rm2_02222014
        x = thickness - 1
        y = kVp
        a1 = 4.1742
        a2 = 1.78924
        b1 = -0.04241
        b2 = 0.04273
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "83cda7ae"  #SanFrancisco Rm4
        x = (thickness) - 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                thickness_calib = [10, 15, 20]
                constant_calib = [84.63934, 82.23884, 80.18645]
                thickness_diff = 1000
                for nn = 1:3
                    if abs(x - thickness_calib[nn]) < thickness_diff
                        thickness_diff = abs(x - thickness_calib[nn])
                        CalibConstant = constant_calib[nn]
                    end
                end
                return CalibConstant
            else
                if y <= 27
                    thickness_calib = [25, 30, 35]
                    constant_calib = [68.52765, 66.88377, 65.41358]
                    thickness_diff = 1000
                    for nn = 1:3
                        if abs(x - thickness_calib[nn]) < thickness_diff
                            thickness_diff = abs(x - thickness_calib[nn])
                            CalibConstant = constant_calib[nn]
                        end
                    end
                else
                    CalibConstant = 62.03676
                end
                return CalibConstant
            end
        else
            a1 = -0.19572
            a2 = 12.69162
            b1 = -0.00281
            b2 = -0.06106
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "H1KRHR84f3b322"  #SanFrancisco Rm5_02222014
        x = (thickness) - 2
        y = (kVp)
        a1 = 12.27149
        a2 = 24.42426
        b1 = -0.01663
        b2 = -0.06336
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == ""  #SanFrancisco Rm7
        x = (thickness) + 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                a1 = 3.08749
                a2 = 14.07335
                b1 = -0.01224
                b2 = -0.05586
                CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
                return CalibConstant
            else
                if abs(x - 40) <= abs(x - 45)
                    CalibConstant = 61.35078
                else
                    CalibConstant = 60.23548
                end
                return CalibConstant
            end
        else
            a1 = -0.0533
            a2 = 12.04488
            b1 = -0.00379
            b2 = -0.05852
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "H1KRHR84f3b8dc"  # Oakland Rm4
        x = (thickness) - 12
        y = (kVp)
        a1 = 15.79098
        a2 = 26.72023
        b1 = -0.01443
        b2 = -0.06941
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84e2dbfc"  # Oakland Rm4
        x = (thickness) - 12
        y = (kVp)
        a1 = 15.79098
        a2 = 26.72023
        b1 = -0.01443
        b2 = -0.06941
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84f3b958"  #Oakland Rm1
        x = (thickness) + 2
        y = (kVp)
        a1 = 10.90211
        a2 = 13.40645
        b1 = -0.02312
        b2 = -0.04199
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84f3b71e"  #OaklandRm2
        x = (thickness) - 1
        y = (kVp)
        a1 = 15.02014
        a2 = 50.31343
        b1 = -0.00666
        b2 = -0.09474
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84f3c1be"  #OaklandRm3
        x = (thickness) + 4
        y = (kVp)
        a1 = -0.55508
        a2 = 4.63406
        b1 = -0.00907
        b2 = -0.00355
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84f3be48"  #OaklandRm5
        x = (thickness) + 1
        y = (kVp)
        a1 = 11.99108
        a2 = 16.82981
        b1 = -0.02112
        b2 = -0.04965
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84f3b906"  #OaklandRm5
        x = (thickness) + 1
        y = (kVp)
        a1 = 11.99108
        a2 = 16.82981
        b1 = -0.02112
        b2 = -0.04965
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84e2909a"  #OaklandRm5_late 2015
        x = (thickness) + 1
        y = (kVp)
        a1 = 11.99108
        a2 = 16.82981
        b1 = -0.02112
        b2 = -0.04965
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end


    if serialNumber == "H1KRHR84f3b00a"  # Richmond Rm1_03252013
        x = (thickness) - 11
        y = (kVp)
        a1 = 13.50227
        a2 = 18.93027
        b1 = -0.01773
        b2 = -0.05579
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84e2dede"  # Richmond Rm2_20131001
        x = (thickness) - 10
        y = (kVp)
        a1 = 16.57312
        a2 = 29.51597
        b1 = -0.01276
        b2 = -0.07426
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84e272ee"  # Richmond Rm2_20131001
        x = (thickness) - 10
        y = (kVp)
        a1 = 16.57312
        a2 = 29.51597
        b1 = -0.01276
        b2 = -0.07426
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR83d894bd"  #WalnutCreek Rm2_20121215
        x = (thickness) - 5
        y = (kVp)
        a1 = 17.97473
        a2 = 30.68309
        b1 = -0.02021
        b2 = -0.05983
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR838bfb7a"  #WalnutCreek Rm1
        x = (thickness) - 12
        y = (kVp)
        a1 = 18.24941
        a2 = 32.07956
        b1 = -0.01995
        b2 = -0.06225
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end


    if serialNumber == "H1KRHR84229908"  # Antioch Rm3
        x = (thickness) - 9
        y = (kVp)
        a1 = 14.34293
        a2 = 22.10439
        b1 = -0.02235
        b2 = -0.04835
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR847083aa"  # Antioch Rm2
        x = (thickness) - 9
        y = (kVp)
        a1 = 15.88117
        a2 = 25.4307
        b1 = -0.02077
        b2 = -0.05554
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84e3005a"  # Antioch Rm2
        x = (thickness) - 9
        y = (kVp)
        a1 = 15.88117
        a2 = 25.4307
        b1 = -0.02077
        b2 = -0.05554
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "207924750210"  #SantaClara Rm4_03102014
        x = (thickness) + 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                if y <= 25
                    CalibConstant = 72.45816
                else
                    thickness_calib = [15, 20, 25]
                    constant_calib = [79.67043, 80.52817, 73.73985]
                    thickness_diff = 1000
                    for nn = 1:3
                        if abs(x - thickness_calib[nn]) < thickness_diff
                            thickness_diff = abs(x - thickness_calib[nn])
                            CalibConstant = constant_calib[nn]
                        end
                    end
                    return CalibConstant
                end
            else
                if y <= 26
                    CalibConstant = 59.06894
                else
                    if abs(x - 35) <= abs(x - 40)
                        CalibConstant = 62.81958
                    else
                        CalibConstant = 61.24175
                    end
                end
                return CalibConstant
            end
        else
            a1 = 6.03368
            a2 = 9.80267
            b1 = -0.01861
            b2 = -0.05276
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "2079247780"  #SantaClara Rm4_Nov 2015
        x = (thickness) + 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                if y <= 25
                    CalibConstant = 72.45816
                else
                    thickness_calib = [15, 20, 25]
                    constant_calib = [79.67043, 80.52817, 73.73985]
                    thickness_diff = 1000
                    for nn = 1:3
                        if abs(x - thickness_calib[nn]) < thickness_diff
                            thickness_diff = abs(x - thickness_calib[nn])
                            CalibConstant = constant_calib[nn]
                        end
                    end
                    return CalibConstant
                end
            else
                if y <= 26
                    CalibConstant = 59.06894
                else
                    if abs(x - 35) <= abs(x - 40)
                        CalibConstant = 62.81958
                    else
                        CalibConstant = 61.24175
                    end
                end
                return CalibConstant
            end
        else
            a1 = 6.03368
            a2 = 9.80267
            b1 = -0.01861
            b2 = -0.05276
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "273316596199"  #SantaClara Rm 1_08062014
        x = (thickness) - 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                thickness_calib = [10, 15, 20]
                constant_calib = [83.82516, 81.36612, 79.20684]
                thickness_diff = 1000
                for nn = 1:3
                    if abs(x - thickness_calib[nn]) < thickness_diff
                        thickness_diff = abs(x - thickness_calib[nn])
                        CalibConstant = constant_calib[nn]
                    end
                end
                return CalibConstant
            else
                if y <= 26
                    CalibConstant = 68.53588
                else
                    if abs(x - 30) <= abs(x - 35)
                        CalibConstant = 65.80543
                    elseif abs(x - 35) <= abs(x - 40)
                        CalibConstant = 64.2097
                    else
                        CalibConstant = 60.87837
                    end
                end
                return CalibConstant
            end
        else
            a1 = -0.07614
            a2 = 7.98597
            b1 = -0.00661
            b2 = -0.03957
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "2079226138200"  #SantaClara Rm 2_07252014
        x = (thickness) - 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                thickness_calib = [10, 15, 20]
                constant_calib = [86.9829, 84.25145, 83.33348]
                thickness_diff = 1000
                for nn = 1:3
                    if abs(x - thickness_calib[nn]) < thickness_diff
                        thickness_diff = abs(x - thickness_calib[nn])
                        CalibConstant = constant_calib[nn]
                    end
                end
                return CalibConstant
            else
                a1 = 4.89214
                a2 = 5.96201
                b1 = -0.02016
                b2 = -0.0328
                CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
                return CalibConstant
            end
        else
            a1 = 24.22642
            a2 = -1.14747
            b1 = -0.02417
            b2 = -0.06299
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "2079247142252"  #SantaClara Rm 2_Nov_2015
        x = (thickness) - 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                thickness_calib = [10, 15, 20]
                constant_calib = [86.9829, 84.25145, 83.33348]
                thickness_diff = 1000
                for nn = 1:3
                    if abs(x - thickness_calib[nn]) < thickness_diff
                        thickness_diff = abs(x - thickness_calib[nn])
                        CalibConstant = constant_calib[nn]
                    end
                end
                return CalibConstant
            else
                a1 = 4.89214
                a2 = 5.96201
                b1 = -0.02016
                b2 = -0.0328
                CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
                return CalibConstant
            end
        else
            a1 = 24.22642
            a2 = -1.14747
            b1 = -0.02417
            b2 = -0.06299
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "273315310659"  #SantaClara Rm3
        x = (thickness) - 2
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                thickness_calib = [10, 15, 20]
                constant_calib = [86.56562, 83.84528, 81.75502]
                thickness_diff = 1000
                for nn = 1:3
                    if abs(x - thickness_calib[nn]) < thickness_diff
                        thickness_diff = abs(x - thickness_calib[nn])
                        CalibConstant = constant_calib[nn]
                    end
                end
                return CalibConstant
            else
                a1 = -0.82803
                a2 = 12.719
                b1 = -0.00229
                b2 = -0.05474
                CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
                return CalibConstant
            end
        else
            a1 = 2.81729
            a2 = 8.79336
            b1 = -0.00927
            b2 = -0.05223
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end

    if serialNumber == "273316754247"  #SantaClara Rm5_20130110
        x = (thickness) - 1
        y = (kVp)
        if anodeMaterial == "Molybdenum"
            if filterMaterial == "Molybdenum"
                thickness_calib = [10, 15, 20]
                constant_calib = [85.52821, 83.43706, 81.35167]
                thickness_diff = 1000
                for nn = 1:3
                    if abs(x - thickness_calib[nn]) < thickness_diff
                        thickness_diff = abs(x - thickness_calib[nn])
                        CalibConstant = constant_calib[nn]
                    end
                end
                return CalibConstant
            else
                if y <= 27
                    thickness_calib = [25, 30, 35]
                    constant_calib = [69.01413, 67.34741, 65.30625]
                    thickness_diff = 1000
                    for nn = 1:3
                        if abs(x - thickness_calib[nn]) < thickness_diff
                            thickness_diff = abs(x - thickness_calib[nn])
                            CalibConstant = constant_calib[nn]
                        end
                    end
                else
                    CalibConstant = 62.57523
                end
                return CalibConstant
            end
        else
            a1 = 3.72318
            a2 = 8.56176
            b1 = -0.01043
            b2 = -0.0538
            CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
            return CalibConstant
        end
    end


    if serialNumber == "H1KRHR84e2f67c"  # Mountain View Rm1_03252013
        x = (thickness) - 12
        y = (kVp)
        a1 = 14.22315
        a2 = 18.00394
        b1 = -0.01848
        b2 = -0.05486
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84f3bbb0"  # Campbell Rm1
        x = (thickness) - 9
        y = (kVp)
        a1 = 14.18175
        a2 = 27.13676
        b1 = -0.01404
        b2 = -0.06832
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84e2a112"  # Pleasanton Rm2
        x = (thickness) - 10
        y = (kVp)
        a1 = 18.17313
        a2 = 29.70029
        b1 = -0.0201
        b2 = -0.06099
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    if serialNumber == "H1KRHR84f3bb5c"  # Pleasanton Rm1
        x = (thickness) - 10
        y = (kVp)
        a1 = 16.36898
        a2 = 24.13466
        b1 = -0.02089
        b2 = -0.05432
        CalibConstant = (a1 * x + a2 * y) * exp(b1 * x + b2 * y)
        return CalibConstant
    end

    return CalibConstant
end

function safe_divide(numerator, denominator)
    if numerator == denominator
        return 1.0f0
    end
    # return denominator == 0f0 ? 1f0 : numerator / denominator
    return denominator == 0.0f0 ? numerator : numerator / denominator
end

function get_IntDen(BAC_img, y, pixel_area)
    sum_pixel_values = sum(BAC_img[y.==1])
    intden = sum_pixel_values * pixel_area
    return intden
end

function get_only_BAC(img_, roi; mask=nothing, r_thd=5000.0f0)
    roi = round.(roi)

    Value_filled = 0.0f0

    # dig holes at roi and background area
    img = deepcopy(img_)
    roi_idxes = findall(isone, roi)
    img[roi_idxes] .= Value_filled
    if !isnothing(mask)
        mask = round.(mask)
        img[mask.==0] .= Value_filled
    end
    else_idx = isnothing(mask) ? (roi .== 0) : (roi .== 0) .& (mask .== 1)
    # some helper vars
    mean_of_else = mean(img[else_idx])

    x_limit, y_limit = size(img)

    x_step = [1, -1, 0, 0, 1, -1, -1, 1]
    y_step = [0, 0, 1, -1, -1, 1, -1, 1]

    num_roi_pixels = size(roi_idxes)[1]
    new_pixel_values = fill(mean_of_else, num_roi_pixels)

    # traversal each pixel in roi
    # @Threads.threads for i = 1 : num_roi_pixels
    for i = 1:num_roi_pixels
        x, y = Tuple(roi_idxes[i])

        # search for 8 directions
        neighbor_values_8 = fill(Value_filled, 8)
        neighbor_distances_8 = fill(r_thd + 1, 8)
        for direction = 1:8
            # start searching for non-hole pixel
            target_x = x + x_step[direction]
            target_y = y + y_step[direction]
            target_pixel = Value_filled
            while abs(target_pixel - Value_filled) < 1.0f-1 && target_x > 0 && target_y > 0 && target_x <= x_limit && target_y <= y_limit
                target_pixel = img[target_x, target_y]

                target_x += x_step[direction]
                target_y += y_step[direction]
            end
            if target_pixel == Value_filled
                # failed to find a neighbor
                # TODO = ?
                # println("here 1")
            else
                # found a neighber
                neighbor_distances_8[direction] = sqrt((target_x - x)^2 + (target_y - y)^2)
                neighbor_values_8[direction] = target_pixel
            end
        end

        # get 4 final distances
        neighbor_values_4 = fill(Value_filled, 4)
        neighbor_distances_4 = fill(r_thd + 1, 4)
        for direction = 1:4
            idx2, idx1 = direction * 2, direction * 2 - 1
            d1, d2 = neighbor_distances_8[idx1], neighbor_distances_8[idx2]
            sum_distance = d1 + d2
            if sum_distance <= r_thd
                v1, v2 = neighbor_values_8[idx1], neighbor_values_8[idx2]
                neighbor_distances_4[direction] = sum_distance
                neighbor_values_4[direction] = (v1 * d2 + v2 * d1) / sum_distance
            else
                # longer than radius threshold
                # TODO = ?
                # println("here 2")
            end
        end
        # pick the value with shortest r
        sorted_indices = sortperm(neighbor_distances_4)
        sorted_distances = neighbor_distances_4[sorted_indices]
        sorted_values = neighbor_values_4[sorted_indices]

        # get final new value for this pixel
        if sorted_distances[1] > r_thd
            # no valid neighber found, fill with mean value
            # new_pixel_values[i] = mean_of_else
            # println("here 3")
        else
            new_pixel_value = sorted_values[1]
            ct = 1.0f0
            for j = 2:4
                if sorted_distances[j] == sorted_distances[1]
                    ct += 1.0f0
                    new_pixel_value += sorted_values[j]
                end
            end
            new_pixel_value /= ct
            new_pixel_values[i] = new_pixel_value
        end
    end

    # fill holes with new values 
    # @Threads.threads for i = 1 : num_roi_pixels
    img = deepcopy(img_)
    for i = 1:num_roi_pixels
        # img[roi_idxes[i]] = new_pixel_values[i]
        img[roi_idxes[i]] = max(img[roi_idxes[i]], new_pixel_values[i])
    end

    return img
end

function get_BAC_mass_data(dcm_data, x, breast_mask, y; isPred=false)
    # read images
    pixel_area = prod(dcm_data[(0x0018, 0x1164)])

    # # dilate pred
    # if isPred
    #     roi = dilate(roi; r = 2)
    # end

    # process roi
    BAC = get_only_BAC(x, y; mask=breast_mask)
    BAC = broadcast(safe_divide, BAC, x)
    BAC = log.(BAC)
    intDen = get_IntDen(BAC, y, pixel_area)

    # calibrate
    serialNumber = dcm_data[(0x0018, 0x1000)]
    thickness = dcm_data[(0x0018, 0x11A0)]
    kVp = dcm_data[(0x0018, 0x0060)]
    anodeMaterial = dcm_data[(0x0018, 0x1191)]
    a, b = CalibConrrection[serialNumber]
    c = GetCalibConstant(serialNumber, thickness, kVp, anodeMaterial)
    mass = intDen / c
    mass = muladd(a, mass, b)

    # key
    LorR = uppercase(dcm_data[(0x0020, 0x0062)])
    V_P = uppercase(dcm_data[(0x0018, 0x5101)])
    curr_key = LorR * " " * V_P

    return mass, BAC, V_P_dict[curr_key]
end

begin
    CalibConrrection = Dict()
    CalibConrrection["H1KRHR838bfb7a"] = [180.83117161003116, -0.391378421662794]
    CalibConrrection["H1KRHR84f3be48"] = [183.38328430470074, -0.40223695064221543]
    CalibConrrection["H1KRHR84e2909a"] = [162.42949974121018, -0.04803082413133412]
    CalibConrrection["H1KRHR84e2a112"] = [181.76630094222094, -0.8753384237514559]
    CalibConrrection["H1KRHR84f3b958"] = [166.6678153842624, 0.1366846770110725]
    CalibConrrection["H1KRHR84f3b8dc"] = [184.30587100548732, -0.42864273887392335]
    CalibConrrection["H1KRHR84e2dbfc"] = [204.63743605555146, -0.9464192215252744]
    CalibConrrection["H1KRHR84229908"] = [169.89325236242118, 0.6866943655396862]
    CalibConrrection["H1KRHR83d894bd"] = [181.93436221947016, -0.3299675337417924]
    CalibConrrection["H1KRHR84f3b71e"] = [180.82428332600915, 0.5246230378140844]
    CalibConrrection["H1KRHR847083aa"] = [165.880821560328, 0.01019528944255794]
    CalibConrrection["H1KRHR84f3bb5c"] = [163.83806521099976, 0.03318350920317485]
    CalibConrrection["H1KRHR84f3b906"] = [194.63999678308454, -1.4382206745885875]
    CalibConrrection["H1KRHR84e3005a"] = [188.93226428520254, -0.4846624036164361]
    CalibConrrection["H1KRHR84f3c1be"] = [194.61330736695635, -2.1640248469210834]
    CalibConrrection["H1KRHR84e2e88e"] = [171.06844723129035, -0.40617526800283077]
    CalibConrrection["H1KRHR84e2e868"] = [159.22272279408628, 0.7123011902402294]
    CalibConrrection["83cda7ae"] = [109.6701351153717, -1.4345873735641654]
    CalibConrrection["H1KRHR84f3b322"] = [178.4794425634932, -0.8969402989466408]
    CalibConrrection["H1KRHR84e2dede"] = [177.4963428292948, -0.1199170535790046]
    CalibConrrection["207924750210"] = [89.13461150640632, -2.209052080364333]
    CalibConrrection["H1KRHR84e2f67c"] = [170.75535631724085, 0.5114904065316838]
    CalibConrrection["H1KRHR84f3b00a"] = [192.46419782057094, -0.6492051326897295]
    CalibConrrection["273315310659"] = [87.2370968685589, -0.3582888600703447]
    CalibConrrection["273316596199"] = [83.69188095254624, -0.5760238941087946]
    CalibConrrection["H1KRHR84f3bbb0"] = [188.4148139197344, -0.7084015877001715]
    CalibConrrection["273316754247"] = [85.48873901078818, 0.2533294643916994]
    CalibConrrection["207924715888"] = [87.5662450946328, -0.35852793163441354]
    CalibConrrection["2079247142252"] = [1.0, 0.0]
    CalibConrrection["2079226138200"] = [88.48478418361002, -0.4984632500587221]
    CalibConrrection["2079247780"] = [83.31149289599844, 0.0015858867954178463]
    CalibConrrection["H1KRHR84e272ee"] = [153.97763843637466, 0.07041160139794743]
end
""

""

In [10]:
num_epochs = 150;

In [4]:
train_dict_path = "JLD2s/train_dict.jld2"
# if !isfile(train_dict_path)
train_dict = Dict()
for rank = 1:4
    @load "JLD2s/train_dl_paths_$(rank).jld2" paths
    for i = size(paths)[1]:-1:1
        splited = split(paths[i], "/")
        sid = splited[end-2]
        f = splited[end]
        vp = split(f, ".")[1]
        train_dict[sid*"."*vp] = (rank, i, f)
    end
end
@save train_dict_path train_dict
# end
# @load train_dict_path train_dict

test_dict_path = "JLD2s/test_dict.jld2"
# if !isfile(test_dict_path)
test_dict = Dict()
for rank = 1:4
    @load "JLD2s/test_dl_paths_$(rank).jld2" paths
    for i = size(paths)[1]:-1:1
        splited = split(paths[i], "/")
        sid = splited[end-2]
        f = splited[end]
        vp = split(f, ".")[1]
        test_dict[sid*"."*vp] = (rank, i, f)
    end
end
@save test_dict_path test_dict
# end
# @load test_dict_path test_dict

In [5]:
train_loaders = []
test_loaders = []
for rank = 1:4
    @load "JLD2s/train_loader_$(rank).jld2" data_loader
    push!(train_loaders, deepcopy(data_loader))
    @load "JLD2s/test_loader_$(rank).jld2" data_loader
    push!(test_loaders, deepcopy(data_loader))
end

In [6]:
train_info = []
test_info = []
for (str, (rank, i, f_name)) in train_dict
    sid, vp = split(str, ".")

    _, y = train_loaders[rank][i]
    if sum(y) > 0
        push!(train_info, (rank, i, sid, f_name, true))
    else
        push!(train_info, (rank, i, sid, f_name, false))
    end
end

for (str, (rank, i, f_name)) in test_dict
    sid, vp = split(str, ".")

    _, y = test_loaders[rank][i]
    if sum(y) > 0
        push!(test_info, (rank, i, sid, f_name, true))
    else
        push!(test_info, (rank, i, sid, f_name, false))
    end
end

@save "JLD2s/train_info.jld2" train_info
@save "JLD2s/test_info.jld2" test_info

# @load "train_info.jld2" train_info
# @load "test_info.jld2" test_info
GC.gc(true)

In [7]:
t = 0.0f0;

In [8]:
for epoch_idx = 1:num_epochs
    # for epoch_idx = 1 : 1
    curr_path = joinpath("training", "outputs", "epoch#$epoch_idx")
    # train
    Threads.@threads for train_info_idx = 1:size(train_info)[1]
        # for train_info_idx = 1 : size(train_info)[1]
        gpu_idx, img_idx, SID, f_name, y_isBAC = train_info[train_info_idx]
        # setup paths
        dcm_path = joinpath(train_dir, SID, f_name * ".dcm")
        ŷ_path = joinpath(curr_path, "train_$(gpu_idx)_$(img_idx)_$(SID).png")
        y_path = joinpath(train_dir, SID, f_name * ".png")
        breast_mask_path = joinpath(train_dir, SID, f_name * ".mask.png")
        resize_info_path = joinpath(train_dir, SID, f_name * "_resize_info.jld2")
        y_BAC_path = joinpath(train_dir, SID, f_name * ".BAC.jld2")
        ŷ_BAC_path = joinpath(curr_path, SID * f_name * ".BAC.jld2")
        @load resize_info_path original_size new_size coords

        # get x, y, ŷ and breast_mask
        dcm_data = dcm_parse(dcm_path)
        x = Float32.(dcm_data[(0x7fe0, 0x0010)])
        y = Float32.(Images.load(y_path))
        ŷ_cropped = Float32.(Images.load(ŷ_path))

        l = (size(ŷ_cropped)[2] - 2) / 3
        start_idx = round(Int, 2 * l + 3)
        ŷ_cropped = ŷ_cropped[:, start_idx:end]

        ŷ = zeros(Float32, (new_size...))
        a, b, c, d = coords
        ŷ[a:b, c:d] = ŷ_cropped
        ŷ = round.(imresize(ŷ, original_size))
        breast_mask = Float32.(Images.load(breast_mask_path))
        if !(size(ŷ) == size(y) == size(breast_mask))
            @info gpu_idx, img_idx, SID, f_name, y_isBAC
            @info size(ŷ)
            @info size(y)
            @info size(breast_mask)
        end
        @assert size(ŷ) == size(y) == size(breast_mask)

        # now we have x, y, ŷ and breast_mask
        if isfile(y_BAC_path)
            @load y_BAC_path label_mass label_BAC label_key
        else
            label_mass, label_BAC, label_key = get_BAC_mass_data(dcm_data, x, breast_mask, y)
            @save y_BAC_path label_mass label_BAC label_key
        end
        pred_mass, pred_BAC, pred_key = get_BAC_mass_data(dcm_data, x, breast_mask, ŷ)
        pred_BAC = ifelse.(pred_BAC .<= t, 0.0f0, 1.0f0)

        # save
        save(joinpath(curr_path, SID * "." * f_name * ".BAC.png"), Gray.(pred_BAC))
        @save ŷ_BAC_path pred_mass pred_key
    end

    # test
    Threads.@threads for test_info_idx = 1:size(test_info)[1]
        # for test_info_idx = 1 : size(test_info)[1]
        gpu_idx, img_idx, SID, f_name, y_isBAC = test_info[test_info_idx]
        # setup paths
        dcm_path = joinpath(test_dir, SID, f_name * ".dcm")
        ŷ_path = joinpath(curr_path, "test_$(gpu_idx)_$(img_idx)_$(SID).png")
        y_path = joinpath(test_dir, SID, f_name * ".png")
        breast_mask_path = joinpath(test_dir, SID, f_name * ".mask.png")
        resize_info_path = joinpath(test_dir, SID, f_name * "_resize_info.jld2")
        y_BAC_path = joinpath(test_dir, SID, f_name * ".BAC.jld2")
        ŷ_BAC_path = joinpath(curr_path, SID * f_name * ".BAC.jld2")
        @load resize_info_path original_size new_size coords

        # get x, y, ŷ and breast_mask
        dcm_data = dcm_parse(dcm_path)
        x = Float32.(dcm_data[(0x7fe0, 0x0010)])
        y = Float32.(Images.load(y_path))
        ŷ_cropped = Float32.(Images.load(ŷ_path))

        l = (size(ŷ_cropped)[2] - 2) / 3
        start_idx = round(Int, 2 * l + 3)
        ŷ_cropped = ŷ_cropped[:, start_idx:end]

        ŷ = zeros(Float32, (new_size...))
        a, b, c, d = coords
        ŷ[a:b, c:d] = ŷ_cropped
        ŷ = round.(imresize(ŷ, original_size))
        breast_mask = Float32.(Images.load(breast_mask_path))


        # now we have x, y, ŷ and breast_mask
        if isfile(y_BAC_path)
            @load y_BAC_path label_mass label_BAC label_key
        else
            label_mass, label_BAC, label_key = get_BAC_mass_data(dcm_data, x, breast_mask, y)
            @save y_BAC_path label_mass label_BAC label_key
        end

        pred_mass, pred_BAC, pred_key = get_BAC_mass_data(dcm_data, x, breast_mask, ŷ)
        pred_BAC = ifelse.(pred_BAC .<= t, 0.0f0, 1.0f0)

        # save
        save(joinpath(curr_path, SID * "." * f_name * ".BAC.png"), Gray.(pred_BAC))
        @save ŷ_BAC_path pred_mass pred_key
    end
    @info epoch_idx
    GC.gc(true)
end

┌ Info: 1
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 2
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 3
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 4
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 5
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 6
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 7
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 8
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 9
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 10
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 11
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 12
└ @ Main /media/molloi-lab/2TB2/BAC ver3/run_interpolation.ipynb:102
┌ Info: 13
└ @ Main /media/molloi-lab/2TB2/BAC ve

In [10]:
GC.gc(true)

In [9]:
# for epoch_idx = 1 : 1
#     curr_path = joinpath(data_dir, "outputs", "epoch#$epoch_idx")
#     # train
#     @Threads.threads for train_info_idx = 1 : size(train_info)[1]
#     # for train_info_idx = 1 : size(train_info)[1]
#         gpu_idx, img_idx, SID, f_name, y_isBAC = train_info[train_info_idx]
#         # setup paths
#         dcm_path = joinpath(train_dir, SID, f_name*".dcm")
#         y_path = joinpath(train_dir, SID, f_name*".png")
#         breast_mask_path =  joinpath(train_dir, SID, f_name*".mask.png")
#         y_BAC_path = joinpath(train_dir, SID, f_name*".BAC.jld2")

#         # get x, y, ŷ and breast_mask
#         dcm_data = dcm_parse(dcm_path)
#         x = Float32.(dcm_data[(0x7fe0, 0x0010)])
#         y = Float32.(Images.load(y_path))
#         breast_mask = Float32.(Images.load(breast_mask_path))

#         # get label data
#         label_mass, label_BAC, label_key = get_BAC_mass_data(dcm_data, x, breast_mask, y)
#         @save y_BAC_path label_mass label_BAC label_key

#     end

#     # test
#     @Threads.threads for test_info_idx = 1 : size(test_info)[1]
#     # for test_info_idx = 1 : size(test_info)[1]
#         gpu_idx, img_idx, SID, f_name, y_isBAC = test_info[test_info_idx]
#         # setup paths
#         dcm_path = joinpath(test_dir, SID, f_name*".dcm")
#         y_path = joinpath(test_dir, SID, f_name*".png")
#         breast_mask_path =  joinpath(test_dir, SID, f_name*".mask.png")
#         y_BAC_path = joinpath(test_dir, SID, f_name*".BAC.jld2")

#         # get x, y, ŷ and breast_mask
#         dcm_data = dcm_parse(dcm_path)
#         x = Float32.(dcm_data[(0x7fe0, 0x0010)])
#         y = Float32.(Images.load(y_path))
#         breast_mask = Float32.(Images.load(breast_mask_path))

#         # get label data
#         label_mass, label_BAC, label_key = get_BAC_mass_data(dcm_data, x, breast_mask, y)
#         @save y_BAC_path label_mass label_BAC label_key

#     end
#     @info epoch_idx
#     GC.gc(true)
# end